 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

**The goal is to predict the last column, whether the donor made a donation in March 2007**, using information about each donor's history. We'll measure success **using recall score** as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [0]:
#!pip install seaborn --upgrade
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.dummy import DummyClassifier

import seaborn as sns

In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [9]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [10]:
df.isnull().sum().sum()

0

## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [20]:
# Finding Baseline Accuracy Manually
X = df.drop(columns='made_donation_in_march_2007')
y = df.made_donation_in_march_2007

import numpy as np
y_pred = np.full(shape=y.shape, fill_value=y.mode())# TODO
baseline_accuracy = accuracy_score(y, y_pred)
print (baseline_accuracy)

0.7620320855614974


In [13]:
# Finding Baseline Accuracy using Sklearn
%%time

# Define an estimator and param_grid
# WHEN DEFINING YOU CAN GIVE IT A NAME OTHERWISE IT WILL USE THE PIPELINE NAME AUTOGEN NAME (name of the function but lowercase)
pipe = make_pipeline(
    DummyClassifier(strategy='most_frequent',random_state=42))

pipe.fit(X, y)

### Get the scores with the MAE Function
## Predict with X_test features and Compare predictions to y_test labels
y_pred = pipe.predict(X)
dummy_score = accuracy_score(y, y_pred)
print('Dummy Classification Score:', dummy_score)

Dummy Classification Score: 0.7620320855614974
CPU times: user 82.9 ms, sys: 3.97 ms, total: 86.8 ms
Wall time: 88.6 ms


What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [18]:
# Finding Baseline Recall Manually
import numpy as np
y_pred = np.full(shape=y.shape, fill_value=y.mode())# TODO
baseline_recall = recall_score(y, y_pred)
print (baseline_recall)

0.0


In [14]:
# Finding Baseline Recall Using Sklearn
%%time
X = df.drop(columns='made_donation_in_march_2007')
y = df.made_donation_in_march_2007

# Define an estimator and param_grid
# WHEN DEFINING YOU CAN GIVE IT A NAME OTHERWISE IT WILL USE THE PIPELINE NAME AUTOGEN NAME (name of the function but lowercase)
pipe = make_pipeline(
    DummyClassifier(strategy='most_frequent',random_state=42))

pipe.fit(X, y)

### Get the scores with the MAE Function
## Predict with X_test features and Compare predictions to y_test labels
y_pred = pipe.predict(X)
dummy_score = recall_score(y, y_pred)
print('Dummy Classification Score:', dummy_score)

Dummy Classification Score: 0.0
CPU times: user 3.96 ms, sys: 1.01 ms, total: 4.97 ms
Wall time: 4.59 ms


## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [0]:
def split(daily):
    # Hold out an "out-of-time" test set, from the last 100 days of data
    X_train, X_test, y_train, y_test = 
    X_train = train.drop(columns='Total')
    y_train = train.Total

    X_test  = test.drop(columns='Total')
    y_test  = test.Total
    
    return X_train, X_test, y_train, y_test

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

Calculate precision

Calculate recall

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 